<a href="https://colab.research.google.com/github/jas293/CS370ForeignWhispers/blob/main/Milestone2Group11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Milestone 2 Group 11

Collaborators: Jill Shah & Dhruvik Patel

In [1]:
%pip install pytube
%pip install youtube_transcript_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00


In [2]:
import os
import re
from pytube import Playlist, YouTube
from pytube.exceptions import AgeRestrictedError
from youtube_transcript_api import YouTubeTranscriptApi

def download_playlist(playlist_url, resolution):
  playlist = Playlist(playlist_url)
  playlist_name = re.sub(r'\W+', '-', playlist.title)

  if not os.path.exists(playlist_name):
    os.mkdir(playlist_name)
    for index, v in enumerate(playlist.videos[:10], start=1):
      try:
        video = YouTube(v.watch_url)
        video_resolution = video.streams.filter(res=resolution).first()
        video_filename = f"{index}. {video_resolution.default_filename}"
        video_path = os.path.join(playlist_name, video_filename)
        video_streams = video.streams.filter(res=resolution)

        if not video_streams:
          highest_resolution_stream = video.streams.get_highest_resolution()
          video_name = highest_resolution_stream.default_filename
          print(
              f"Downloading {video_name} in {highest_resolution_stream.resolution}")
          highest_resolution_stream.download(filename=video_path)


        else:
          video_stream = video_streams.first()
          video_name = video_stream.default_filename
          print(
              f"Downloading video for {video_name} in {resolution}")
          video_stream.download(filename="video.mp4")


          audio_stream = video.streams.get_audio_only()
          print(
              f"Downloading audio for {video_name}")
          audio_stream.download(filename="audio.mp4")
          os.system(
              "ffmpeg -y -i video.mp4 -i audio.mp4 -c:v copy -c:a aac final.mp4 -loglevel quiet -stats")
          os.rename("final.mp4", video_path)
          os.remove("video.mp4")
          os.remove("audio.mp4")

          srt = YouTubeTranscriptApi.get_transcript(video.video_id)
          caption_file = f"{video_path}.srt"
          with open(caption_file, 'w') as f:
            for line in srt:
              end=line["start"]+line["duration"]
              f.write(str(line["start"]) + " -- " + str(end) + "\n")
              f.write(line["text"] + "\n")

      except AgeRestrictedError:
                print(f"Video {v.title} is age-restricted and has been skipped.")
                continue


playlist_url = "https://www.youtube.com/playlist?list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL"
resolution = "720p"
download_playlist(playlist_url, resolution)

Video President Joe Biden: The 2023 60 Minutes Interview is age-restricted and has been skipped.


In [3]:
pip install moviepy


In [4]:
mkdir audio

In [5]:
import os
from moviepy.editor import *

# Directory containing your videos
video_dir = '/content/Interviews'
# Directory to save audio files
audio_dir = '/content/audio'

# Ensure the audio directory exists
os.makedirs(audio_dir, exist_ok=True)

def process_video(video_path):
    # Load the video file
    video = VideoFileClip(video_path)

    # Get the audio from the video
    audio = video.audio

    # Build the audio file path
    audio_file_name = os.path.basename(video_path).replace('.mp4', '.mp3')
    audio_file_path = os.path.join(audio_dir, audio_file_name)

    # Write the audio to file
    audio.write_audiofile(audio_file_path)

# Get all video files in the video directory
video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]

# Process each video file
for video_file in video_files:
    video_path = os.path.join(video_dir, video_file)
    process_video(video_path)


MoviePy - Writing audio in /content/audio/4. Rich Paul The 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/audio/5. Godfather of AI Geoffrey Hinton The 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/audio/2. Pink The 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/audio/1. Vice President Kamala Harris The 2023 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/audio/6. Gen Mark Milley The 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/audio/10. Denzel Washington  60 Minutes Archive.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/audio/8. Deion Sanders The 2023 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/audio/7. Attorney General Merrick Garland The 60 Minutes Interview.mp3


MoviePy - Done.
MoviePy - Writing audio in /content/audio/9. Volodymyr Zelenskyy The 2023 60 Minutes Interview.mp3


MoviePy - Done.


In [6]:
! pip install git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ascvixq6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-ascvixq6
  Resolved https://github.com/openai/whisper.git to commit b38a1f20f4b23f3f3099af2c3e0ca95627276ddf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.1/153.1 kB 20.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
   

In [7]:
import whisper
import os

def convert_audio_to_text_whisper_and_save(audio_directory):
    """
    This function will take the directory containing audio files, load each audio file,
    run Whisper's speech recognition on it, and save the transcribed text into a text file
    with the same name as the audio file in the same directory.
    """
    # Initialize the model
    model = whisper.load_model("base.en")

    # List all the audio files in the directory
    audio_files = [f for f in os.listdir(audio_directory) if f.endswith('.mp3')]

    # Process each audio file
    for audio_file in audio_files:
        audio_path = os.path.join(audio_directory, audio_file)
        # Load the audio file
        result = model.transcribe(audio_path)
        # Build the text file path
        text_file_path = audio_path.replace('.mp3', '.txt')
        # Write the transcription to the text file
        with open(text_file_path, 'w') as text_file:
            text_file.write(result['text'])

convert_audio_to_text_whisper_and_save('/content/audio')

100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 30.9MiB/s]
